In [9]:
import pandas as pd 
from urllib.request import urlopen
from dotenv import dotenv_values
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.postgresql import JSON as postgres_json
import pandas as pd
import requests
import json
from datetime import datetime
import time
import pprint

# Reading Session keys

In [10]:
session_key_list = pd.read_csv('session_keys.csv')

In [11]:
for i in session_key_list['session_key']:
    response = urlopen(f'https://api.openf1.org/v1/weather?session_key={i}')
    data = json.loads(response.read().decode('utf-8'))
    if i == 7953:
        df_weather = pd.DataFrame(data)
    else:
        hold = pd.DataFrame(data)
        df_weather = pd.concat([df_weather,hold])
        time.sleep(0.5)
    print(f'Session {i} extracted.')
print('Done')


Session 7953 extracted.
Session 7779 extracted.
Session 7787 extracted.
Session 9070 extracted.
Session 9078 extracted.
Session 9094 extracted.
Session 9102 extracted.
Session 9110 extracted.
Session 9118 extracted.
Session 9126 extracted.
Session 9133 extracted.
Session 9141 extracted.
Session 9149 extracted.
Session 9157 extracted.
Session 9165 extracted.
Session 9173 extracted.
Session 9221 extracted.
Session 9213 extracted.
Session 9181 extracted.
Session 9205 extracted.
Session 9189 extracted.
Session 9197 extracted.
Session 9472 extracted.
Session 9480 extracted.
Session 9488 extracted.
Session 9496 extracted.
Session 9673 extracted.
Session 9507 extracted.
Session 9515 extracted.
Session 9523 extracted.
Session 9531 extracted.
Session 9539 extracted.
Session 9550 extracted.
Session 9558 extracted.
Session 9566 extracted.
Session 9574 extracted.
Session 9582 extracted.
Session 9590 extracted.
Session 9598 extracted.
Session 9606 extracted.
Session 9617 extracted.
Session 9625 ext

In [12]:
df_weather

,date,session_key,wind_direction,meeting_key,wind_speed,rainfall,track_temperature,air_temperature,humidity,pressure
0,2023-03-05T14:01:47.286000+00:00,7953,176,1141,1.2,0,35.1,29.8,19.0,1016.5
1,2023-03-05T14:02:47.301000+00:00,7953,182,1141,1.2,0,35.0,29.7,19.0,1016.5
2,2023-03-05T14:03:47.300000+00:00,7953,156,1141,1.1,0,34.9,29.7,19.0,1016.5
3,2023-03-05T14:04:47.314000+00:00,7953,201,1141,0.8,0,34.9,29.6,19.0,1016.5
4,2023-03-05T14:05:47.297000+00:00,7953,219,1141,0.8,0,34.8,29.6,19.0,1016.5
...,...,...,...,...,...,...,...,...,...,...
148,2025-12-07T14:35:07.945000+00:00,9839,88,1276,1.7,0,28.7,26.0,66.0,1017.4
149,2025-12-07T14:36:07.943000+00:00,9839,128,1276,1.4,0,28.9,26.0,66.0,1017.3
150,2025-12-07T14:37:07.949000+00:00,9839,101,1276,0.6,0,28.9,25.9,66.0,1017.4
151,2025-12-07T14:38:07.948000+00:00,9839,104,1276,1.2,0,29.0,25.9,66.0,1017.4


In [13]:
config = dotenv_values()

pg_user = config['POSTGRES_USER']
pg_pass = config['POSTGRES_PASS']
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']

url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'
engine = create_engine(url, echo=False)

In [14]:
df_weather.columns

Index(['date', 'session_key', 'wind_direction', 'meeting_key', 'wind_speed',
       'rainfall', 'track_temperature', 'air_temperature', 'humidity',
       'pressure'],
      dtype='object')

In [15]:
dtype_dict = {
    'date': types.DateTime,
    'session_key': types.Integer,
    'wind_direction': types.Float,
    'meeting_key': types.Integer,
    'wind_speed': types.Float,
    'rainfall': types.Float,
    'track_temperature': types.Float,
    'air_temperature': types.Float,
    'humidity': types.Float,
    'pressure': types.Float
}

In [16]:
df_weather.to_sql(name = 'raw_weather', 
                       con = engine, 
                       schema = pg_schema, 
                       if_exists='replace', 
                       dtype=dtype_dict,
                       index=False
                      )

916